<a href="https://colab.research.google.com/github/ngdeva99/ACM-ICPC-Algorithms/blob/master/Codes/Data%20Processing/Sentence_Dataset_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**RP Dataset Preparation**

*   CSV Generation from .wav Files.
*   RP Generation from CSV
*   Energy Distribution Visualization






---



## **CSV Generation from .wav files**


###**Imports**

In [ ]:
import os
import numpy as np
import librosa
import pandas as pd

###**Librosa Code to Generate CSV**

In [ ]:
def gernerate_csv_files(speaker):
  #Assigning the the root directory of the dataset
  root_dir="C:/Users/Navein Kumar/Downloads/Compressed/6speakers_wav/6speakers_wav/" + speaker +"/"
  for dir_, _, files in os.walk(root_dir):
      for file_name in files:
          if not file_name == '.DS_Store' and file_name[-5:]!='ipynb':
              rel_dir = os.path.relpath(dir_, root_dir)
              rel_file = os.path.join(rel_dir, file_name)
              file_no_ext,ext=os.path.splitext(file_name)

              y, sr = librosa.load(root_dir + rel_file)


              np.savetxt(root_dir+rel_dir+"/"+file_no_ext+".csv",y.transpose(),delimiter=",")


###**Invoking CSV Generation Code for all Speakers**

In [ ]:
speakers=["Anusuya","Divya","Giri","Robert"]
for speaker in speakers:
  generare_csv_file(speaker) #-> uncommet it u havent generated

##**RP Generation from CSV**

###**Imports**

In [ ]:
from matplotlib import pyplot as plt
from IPython.display import display_html

import matplotlib.image as mpimg
import IPython.display as ipd
from sklearn.preprocessing import MinMaxScaler
import pyts
from pyts.image import RecurrencePlot

###**Optimization Code to Restart Kernel at Regular Intervals for Faster Generation Rate**

In [ ]:
def restartkernel() :
  display_html("<script>Jupyter.notebook.kernel.restart(); setTimeout(function(){IPython.notebook.execute_all_cells();}, 10000)</script>",raw=True)

###**RP Generation Code**

In [ ]:
def generate_rp(frame, dimension, threshold, percentage, file_name, count,time_delay):
    
    #Preparing RP Data as an numpy array
    data_rp = []
    data_rp.append(frame.values.reshape(1,-1))
    data_rp.append(frame.values.reshape(1,-1))
    data_rp = np.asarray(data_rp)

    
    # Recurrence plot transformation
    #X_rp1 = RecurrencePlot(dimension=3, time_delay=1,threshold=None, percentage=0).fit_transform(data_rp[0])[0]
    X_rp1 = RecurrencePlot(dimension=dimension, time_delay=time_delay,threshold=threshold, percentage=percentage).fit_transform(data_rp[0])[0]

    #Applying cmap = binary to generate grayscale image.
    imgplot = plt.imshow(X_rp1, cmap='binary', origin='lower')

    fig1 = plt.gcf()
    #plt.show() -> uncomment to visualize the RP Plot Created
    plt.draw()
    fig1.savefig(file_name+'_RP'+str(count)+'.png')

###**Invoking RP Generation Code**

In [ ]:

def compute_total_energy_average(data_frame):    
  array=data_frame.to_numpy()
  energy=0

  for i in range(len(array)):
      energy+=pow(array[i][0],2)

  return energy  
def remove_files(files):
  for i in files:
    os.remove(i)


    
root_dir="C:/Users/Navein Kumar/Downloads/Compressed/6speakers_wav/6speakers_wav/Anusuya/"
file_names=[]
for dir_, _, files in os.walk(root_dir):
  for file_name in files:
    if len(file_names)>15:
      #removing csv files after RP Generation
      remove_files(file_names)
      file_names=[]
      #Will Restart the kernel for every 15 RP's Generated
      print("gonna restart kernel")
      restartkernel()
      print(file_name)

    #Applying this condition to eliminate junk and unsupported files
    if  file_name!= '.DS_Store' and file_name[-4:]!= '.wav' and file_name[-4:]!=".png" and file_name[-5:]!='ipynb' :
      rel_dir = os.path.relpath(dir_, root_dir)
      rel_file = os.path.join(rel_dir, file_name)
      file_no_ext,ext=os.path.splitext(file_name)
          
      #Readind CSV file
      df = pd.read_csv(root_dir + rel_file, index_col=None, header=None)

      count = 0 #Reinitializing count for a fresh batch
      end = df.shape[0] #Reading the no of rows in the CSV file

      #This computed the total energy of the entire data file
      total_energy_average=compute_total_energy_average(df)

      #Generating RP Function for every 600 non overallaping samples (i.e, for each 600 rows of the csv file)
      for i in range(0, end, 600):
        f = i + 600
        frame = df[i:f]

        #Computing energy of the present frame of size 600 rows
        sample_average=compute_total_energy_average(frame)

        #This condition eliminates the unwanted low energy frames at the start & end of .wav files (Treshold -0.1)
        if sample_average>0.1:
          #Calling RP Generation function with following parameters (m -> 2, tau->6, and 0 Threshold)
          generate_rp(frame, 2, None, 0, file_no_ext, count,6) 
          count +=1 
      
      file_names.append(root_dir+file_name)

## **Energy Distribution Visualization**

###**Imports**

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd


In [ ]:
!gdown --id 1gMR5uPC6NpBcwU25n-BIlSDhVfwi0oDo
!unzip '6speakers_wav (1).zip'

###**Plotting Energy Distribution Graph**

In [ ]:
root_dir="/content/6speakers_wav (1)/6speakers_wav (1)/"
csv_plots=[]

#using Anusuya for Energy Visualization 
for dir_, _, files in os.walk(root_dir+"Anusuya/"):
  for file_name in files:
    inference_sub=[]
    if not file_name == '.DS_Store' and file_name[-4:]!= '.wav':
      rel_dir = os.path.relpath(dir_, root_dir)
      rel_file = os.path.join(rel_dir, file_name)
      file_no_ext,ext=os.path.splitext(file_name)

      df = pd.read_csv(root_dir + rel_file, index_col=None, header=None)
      csv_plots.append(df)

#Defining Plot Size and Divisions for visualization
figure, axes = plt.subplots(nrows=5, ncols=3,sharey=True)
figure.set_size_inches(18.5, 10.5)


count=0
for i in range(5):
  for j in range(3):
    y=csv_plots[count]

    #Data Values for Plot
    x=[i for i in range(0,len(csv_plots[count]))]
    axes[i, j].plot(x, y)
    count+=1


In [ ]:
#Energy Distribution Visualization
figure, axes = plt.subplots(nrows=1, ncols=1,sharey=True)
figure.set_size_inches(30, 30)

plt.yticks(np.arange(0,6.5,0.5))

count=0
x=[i for i in range(600)]
for i in range(7):
    
  for j in range(2):
      
    for k in range(len(inference[count])):
      y=[inference[count][k]]*600
      axes.plot(x, y)
      
      count+=1


In [ ]:
#CSV File - Based Visualization

figure, axes = plt.subplots(nrows=3, ncols=5,sharey=True)
figure.set_size_inches(30, 30)

plt.yticks(np.arange(0,6.5,0.5))



count=0
for i in range(3):
    
    for j in range(5):
        x=[i for i in range(len(inference[count]))]
        y=inference[count]
        
        axes[i,j].plot(x,y)
        
     
        count+=1
        